In [5]:
!pip3 install customtkinter
!pip install python-barcode[images]
!pip install pyinstaller
!pip install mysql-connector-python
!pip install PyMySQL
!pip install python-dotenv

In [ ]:
import tkinter as tk
from tkinter import *
from tkinter import messagebox
import sqlite3
from PIL import Image, ImageTk
import os
import tkinter as tk
from PIL import Image, ImageTk
from tkinter import Frame
from tkinter import ttk
import customtkinter
from barcode import Code128
from barcode.writer import ImageWriter
import os
import mysql.connector
from mysql.connector import errors
import mysql.connector
from mysql.connector import errors
import os 
from datetime import datetime
from dotenv import load_dotenv
import os

load_dotenv()
db_host = os.getenv("DB_HOST")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_name = os.getenv("DB_NAME")

conn = mysql.connector.connect(
    host=os.getenv("DB_HOST","localhost"),
    user=os.getenv("DB_USER"),
    passwd=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB_NAME")
)
cursor = conn.cursor()

os.makedirs("barcodes", exist_ok=True)
cursor.execute('''
               
CREATE TABLE IF NOT EXISTS items (
    code VARCHAR(255) PRIMARY KEY,
    name TEXT,
    type TEXT,
    color TEXT,
    size TEXT,
    location TEXT,
    quantity INT,
    price FLOAT,
    sold INT DEFAULT 0,
    description TEXT
)
''')
cursor.execute('''CREATE TABLE IF NOT EXISTS sales (id INT PRIMARY KEY AUTO_INCREMENT, product_code VARCHAR(50) NOT NULL, product_name VARCHAR(100) NOT NULL, product_type VARCHAR(100), product_color VARCHAR(50), customer_name VARCHAR(100) NOT NULL, mobile_number VARCHAR(20), total_amount DECIMAL(10, 2) NOT NULL, paid_amount DECIMAL(10, 2) DEFAULT 0.00, payment_method VARCHAR(50), transaction_id VARCHAR(100), fully_paid ENUM('Yes', 'No') DEFAULT 'No', payment_status ENUM('Pending', 'Partial', 'Paid') DEFAULT 'Pending', last_payment_date DATE, customer_note TEXT, date_purchased DATE, created_at DATETIME DEFAULT CURRENT_TIMESTAMP, updated_at DATETIME DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP);''')


def open_add_window():
    add_window = customtkinter.CTkToplevel()
    add_window.transient(root)
    add_window.grab_set() 
    add_window.configure(fg_color="white") 
    add_window.geometry("1200x650+220+130")  
    add_window.focus_force()
    add_window.title("Add Product")
    var_code = StringVar()
    var_name = StringVar()
    var_type = StringVar()
    var_color = StringVar()
    var_size = StringVar()
    var_location = StringVar()
    var_quant = StringVar()
    var_price = StringVar()
    var_sold = StringVar()
    var_desc = StringVar()
    #r1
    # Row 1 - Labels
    customtkinter.CTkLabel(add_window, text='Product Code \n كود المنتج', font=("Segoe UI Semibold", 15), text_color="#2C3E50").place(x=50, y=150)
    customtkinter.CTkLabel(add_window, text='Product Name \n اسم المنتج', font=("Segoe UI Semibold", 15), text_color="#2C3E50").place(x=380, y=150)
    customtkinter.CTkLabel(add_window, text='Product Type \n نوع المنتج', font=("Segoe UI Semibold", 15), text_color="#2C3E50").place(x=720, y=150)

    # Row 1 - Entries
    customtkinter.CTkEntry(add_window, textvariable=var_code, font=("Segoe UI", 14),
        width=190, height=35, fg_color="#ECF0F1", text_color="#34495E", corner_radius=8).place(x=180, y=160)
    customtkinter.CTkEntry(add_window, textvariable=var_name, font=("Segoe UI", 14),
        width=190, height=35, fg_color="#ECF0F1", text_color="#34495E", corner_radius=8).place(x=520, y=160)
    customtkinter.CTkEntry(add_window, textvariable=var_type, font=("Segoe UI", 14),
        width=190, height=35, fg_color="#ECF0F1", text_color="#34495E", corner_radius=8).place(x=850, y=160)

    # Row 2 - Labels
    customtkinter.CTkLabel(add_window, text='Product Color \n لون المنتج', font=("Segoe UI Semibold", 15), text_color="#2C3E50").place(x=50, y=210)
    customtkinter.CTkLabel(add_window, text='Product Size \n حجم المنتج', font=("Segoe UI Semibold", 15), text_color="#2C3E50").place(x=380, y=210)
    customtkinter.CTkLabel(add_window, text='Location \n المستودع', font=("Segoe UI Semibold", 15), text_color="#2C3E50").place(x=720, y=210)

    # Row 2 - Entries
    customtkinter.CTkEntry(add_window, textvariable=var_color, font=("Segoe UI", 14),
        width=190, height=35, fg_color="#ECF0F1", text_color="#34495E", corner_radius=8).place(x=180, y=220)
    customtkinter.CTkEntry(add_window, textvariable=var_size, font=("Segoe UI", 14),
        width=190, height=35, fg_color="#ECF0F1", text_color="#34495E", corner_radius=8).place(x=520, y=220)
    customtkinter.CTkEntry(add_window, textvariable=var_location, font=("Segoe UI", 14),
        width=190, height=35, fg_color="#ECF0F1", text_color="#34495E", corner_radius=8).place(x=850, y=220)
    customtkinter.CTkLabel(add_window, text='Quantity \n كمبة المنتج', font=("Segoe UI Semibold", 15), text_color="#2C3E50").place(x=50, y=270)
    customtkinter.CTkLabel(add_window, text='Product Price \n سعر المنتج', font=("Segoe UI Semibold", 15), text_color="#2C3E50").place(x=380, y=270)
    customtkinter.CTkLabel(add_window, text='Sold \n مباع', font=("Segoe UI Semibold", 15), text_color="#2C3E50").place(x=720, y=270)
    customtkinter.CTkEntry(add_window, textvariable=var_quant, font=("Segoe UI", 14),
        width=190, height=35, fg_color="#ECF0F1", text_color="#34495E", corner_radius=8).place(x=180, y=280)
    customtkinter.CTkEntry(add_window, textvariable=var_price, font=("Segoe UI", 14),
        width=190, height=35, fg_color="#ECF0F1", text_color="#34495E", corner_radius=8).place(x=520, y=280)
    customtkinter.CTkEntry(add_window, textvariable=var_sold, font=("Segoe UI", 14),
        width=190, height=35, fg_color="#ECF0F1", text_color="#34495E", corner_radius=8).place(x=850, y=280)

    # Row 4 - Description
    customtkinter.CTkLabel(add_window, text='Description \n المنتج وصف', font=("Segoe UI Semibold", 15), text_color="#2C3E50").place(x=50, y=330)
    customtkinter.CTkEntry(add_window, textvariable=var_desc, font=("Segoe UI", 14),
        width=860, height=100, fg_color="#ECF0F1", text_color="#34495E", corner_radius=12).place(x=180, y=340)





    #BUTTS

    btn_font = ("Segoe UI", 12, "bold")
    def add_to_db():
        code = var_code.get().strip()
        name = var_name.get().strip()
        type_ = var_type.get().strip()
        color = var_color.get().strip()
        size = var_size.get().strip()
        location = var_location.get().strip()
        quantity = var_quant.get().strip()
        price = var_price.get().strip()
        sold = var_sold.get().strip()
        description = var_desc.get().strip()


        if not all([code, name, type_, color, size, location, quantity, price, sold]):
            messagebox.showerror(
    "خطأ - Error",
    "Please fill in all fields.\n"
    "يرجى تعبئة جميع الحقول .",parent=root
)
            return

        try:
            with mysql.connector.connect(host=os.getenv("DB_HOST","localhost"),user=os.getenv("DB_USER"),passwd=os.getenv("DB_PASSWORD"),database=os.getenv("DB_NAME")) as conn:
                cursor = conn.cursor()
              
                cursor.execute("""
                    INSERT INTO items 
                    (code, name, type, color, size, location, quantity, price, sold, description) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, (
                    code,
                    name,
                    type_,
                    color,
                    size,
                    location,
                    int(quantity),
                    float(price),
                    int(sold),
                    description
                ))

                conn.commit()

               
                barcode_dir = "barcodes"
                os.makedirs(barcode_dir, exist_ok=True)
                barcode_path = os.path.join(barcode_dir, f"{name}")

                code128 = Code128(code, writer=ImageWriter())
                code128.save(barcode_path)

                messagebox.showinfo(
    "نجاح - Success",
    "Item added to database.\n"
    "تمت إضافة العنصر إلى قاعدة البيانات.",parent=root
)
                add_window.destroy()

        except errors.IntegrityError:
            messagebox.showerror(
    "خطأ - Error",
    "Item with this code already exists.\n"
    "يوجد بالفعل عنصر بهذا الرمز."
)
        except errors.OperationalError as e:
            messagebox.showerror(
    "خطأ في قاعدة البيانات - Database Error",
    f"MySQL Operational Error:\n{e}\n"
    f"خطأ في عمليات MySQL: {e}"
)
        except ValueError:
            messagebox.showerror(
    "خطأ - Error",
    "Please enter valid numbers for Quantity, Price, and Sold.\n"
    "يرجى إدخال أرقام صحيحة للكمية والسعر والمباع.",parent=root
)
    def update_item():
        code = var_code.get().strip()
        if not code:
            messagebox.showerror(
    "خطأ - Error",
    "Please enter the Product Code to update.\n"
    "يرجى إدخال رمز المنتج للتحديث."
)
            return

        updates = {}
        if var_name.get().strip(): updates["name"] = var_name.get().strip()
        if var_type.get().strip(): updates["type"] = var_type.get().strip()
        if var_color.get().strip(): updates["color"] = var_color.get().strip()
        if var_size.get().strip(): updates["size"] = var_size.get().strip()
        if var_location.get().strip(): updates["location"] = var_location.get().strip()
        if var_quant.get().strip(): updates["quantity"] = var_quant.get().strip()
        if var_price.get().strip(): updates["price"] = var_price.get().strip()
        if var_sold.get().strip(): updates["sold"] = var_sold.get().strip()
        if var_desc.get().strip(): updates["description"] = var_desc.get().strip()

        if not updates:
            messagebox.showwarning("Warning", "No fields to update.")
            return


        try:
            with mysql.connector.connect(host=os.getenv("DB_HOST","localhost"),user=os.getenv("DB_USER"),passwd=os.getenv("DB_PASSWORD"),database=os.getenv("DB_NAME")) as conn:
                cursor = conn.cursor()

                
                set_clause = ", ".join(f"{key} = %s" for key in updates.keys())
                values = list(updates.values())
                values.append(code)  #

                query = f"UPDATE items SET {set_clause} WHERE code = %s"
                cursor.execute(query, values)

                if cursor.rowcount == 0:
                    messagebox.showerror(
    "خطأ - Error",
    "No item found with the given Product Code.\n"
    "لم يتم العثور على أي عنصر برمز المنتج المدخل."
)
                else:
                    conn.commit()
                    messagebox.showinfo(
    "نجاح - Success",
    "Item updated successfully.\n"
    "تم تحديث العنصر بنجاح."
)
                    add_window.destroy()

        except errors.OperationalError as e:
            messagebox.showerror(
    "خطأ في قاعدة البيانات - Database Error",
    f"MySQL error:\n{e}\n"
    f"خطأ في MySQL:\n{e}"
)
        except ValueError:
            messagebox.showerror(
    "خطأ - Error",
    "Invalid data entered.\n"
    "تم إدخال بيانات غير صحيحة."
)

    def delete_item():
        code = var_code.get().strip()
        if not code:
            messagebox.showerror(
    "خطأ - Error",
    "Please enter the Product Code to delete.\n"
    "يرجى إدخال رمز المنتج للحذف."
)
            return

        confirm = messagebox.askyesno(
    "تأكيد الحذف - Confirm Delete",
    f"Are you sure you want to delete item with code '{code}'?\n"
    f"هل أنت متأكد أنك تريد حذف العنصر برمز '{code}'؟"
)
        if not confirm:
            return

        try:
            with mysql.connector.connect(host=os.getenv("DB_HOST","localhost"),user=os.getenv("DB_USER"),passwd=os.getenv("DB_PASSWORD"),database=os.getenv("DB_NAME")) as conn:
                cursor = conn.cursor()
                cursor.execute("DELETE FROM items WHERE code = %s", (code,))
                
                if cursor.rowcount == 0:
                    messagebox.showerror(
    "خطأ - Error",
    "No item found with that Product Code.\n"
    "لم يتم العثور على أي عنصر برمز المنتج هذا."
)
                else:
                    conn.commit()
                    messagebox.showinfo(
    "نجاح - Success",
    f"Item with code '{code}' has been deleted.\n"
    f"تم حذف العنصر برمز '{code}'."
)
                    
        except errors.OperationalError as e:
            messagebox.showerror("Database Error", f"Database is locked or busy: Please contact 0787655553 :\n{e}")
    customtkinter.CTkLabel(
        add_window, 
        text="   منتج حذف/تعديل/اضافة", 
        font=("Segoe UI Semibold", 20), 
        text_color="white", 
        fg_color="#4c74af", 
        corner_radius=8, width=1300, height=40
    ).place(x=50, y=100)

    # Save Button
    customtkinter.CTkButton(
        master=add_window,
        text="Save / حفظ",
        command=add_to_db,
        font=btn_font,
        width=110,
        height=32,
        corner_radius=10,
        fg_color="#27AE60",
        hover_color="#219150",
        text_color="white",
        cursor="hand2"
    ).place(x=200, y=450)

    # Update Button
    customtkinter.CTkButton(
        master=add_window,
        text="Update / تعديل",
        font=btn_font,
        width=110,
        height=32,
        corner_radius=10,
        fg_color="#2980B9",
        hover_color="#2471A3",
        text_color="white",
        cursor="hand2",command=update_item
    ).place(x=350, y=450)

    # Delete Button
    customtkinter.CTkButton(
        master=add_window,
        text="Delete / حذف",
        font=btn_font,
        width=110,
        height=32,
        corner_radius=10,
        fg_color="#C0392B",     
        hover_color="#A93226",
        text_color="white",
         command=delete_item,
        cursor="hand2"
    ).place(x=500, y=450)



            
def products():
    style = ttk.Style()
    style.map("Treeview", background=[("selected", "#cce5ff")])
    
    win = tk.Toplevel()
    win.title("Products - Style Office")
    win.config(bg="white")
    win.geometry("1200x650+220+130")
    win.focus_force()
    
    conn = mysql.connector.connect(host=os.getenv("DB_HOST","localhost"),user=os.getenv("DB_USER"),passwd=os.getenv("DB_PASSWORD"),database=os.getenv("DB_NAME"))
    cursor = conn.cursor()

    # Search Frame
    SearchFrame = tk.LabelFrame(win, text="Search Product Details / المنتج تفاصيل عن البحث", bg="white",
                                font=("goudy old style", 12, "bold"), bd=2, relief="ridge")
    SearchFrame.place(x=550, y=20, width=650, height=70)

    # Combo Box
    cc = ttk.Combobox(SearchFrame,
                      values=("Select", "All / الكل", "Code / رقم المنتج", "Product Name / اسم المنتج", "Type / النوع", "Location / الموقع"),
                      state="readonly", justify="center", font=("Segoe UI", 15))
    cc.place(x=10, y=10, width=180)

    # Search Entry
    search_txt = customtkinter.CTkEntry(
        master=SearchFrame,
        placeholder_text="ادخل المطلوب",
        font=("Segoe UI", 15),
        height=30,
        width=150,
        corner_radius=25,
        fg_color="#2C3E50",
        text_color="white",
        placeholder_text_color="#BDC3C7"
    )
    search_txt.place(x=155, y=3)

    # Treeview frame and setup
    frame = Frame(win, bd=2, relief="ridge", bg="white")
    frame.place(x=50, y=140, width=1800, height=850)

    scrolly = Scrollbar(frame, orient="vertical")
    scrollx = Scrollbar(frame, orient="horizontal")

    tree = ttk.Treeview(
        frame,
        columns=("code", "name", "type", "color", "size", "location", "quantity", "price", "sold", "description"),
        show="headings",
        yscrollcommand=scrolly.set,
        xscrollcommand=scrollx.set
    )
    tree.pack(fill="both", expand=True)
    scrolly.config(command=tree.yview)
    scrollx.config(command=tree.xview)
    scrolly.pack(side="right", fill="y")
    scrollx.pack(side="bottom", fill="x")
    headers = {
        "code": "Code / رقم المنتج",
        "name": "Product Name / اسم المنتج",
        "type": "Type / النوع",
        "color": "Color / اللون",
        "size": "Size / الحجم",
        "location": "Location / الموقع",
        "quantity": "Quantity / الكمية",
        "price": "Price / السعر",
        "sold": "Sold / مباع",
        "description": "Description / وصف"
    }

    for col in headers:
        tree.heading(col, text=headers[col])
        if col == "description":
            tree.column(col, width=250, anchor="w")
        elif col in ["code", "price", "sold", "quantity"]:
            tree.column(col, width=80, anchor="center")
        else:
            tree.column(col, width=120, anchor="center")

    tree.tag_configure('oddrow', background="white")
    tree.tag_configure('evenrow', background="#f2f2f2")

    def show_full_description(event):
        selected_item = tree.focus()
        if not selected_item:
            return
        data = tree.item(selected_item, "values")
        if len(data) >= 10:
            description = data[9]
            tk.messagebox.showinfo("Full Description - الوصف كامل", description)

    tree.bind("<Double-1>", show_full_description)
    def search():
        field_map = {
            "code / رقم المنتج": "code",
            "product name / اسم المنتج": "name",
            "type / النوع": "type",
            "location / الموقع": "location"
        }

        selected = cc.get().strip().lower()
        keyword = search_txt.get().strip()

        if selected == "select" or (selected != "all / الكل" and not keyword):
            messagebox.showwarning("تحذير - Warning",
                                   "Please select a field and enter a keyword.\n"
                                   "يرجى اختيار حقل وإدخال كلمة مفتاحية.",
                                   parent=win)
            return

        try:
            conn= mysql.connector.connect(host=os.getenv("DB_HOST","localhost"),user=os.getenv("DB_USER"),passwd=os.getenv("DB_PASSWORD"),database=os.getenv("DB_NAME"))
            cursor = conn.cursor()

            if selected == "all / الكل":
                cursor.execute("SELECT * FROM items")
            else:
                matched_field = None
                for k in field_map:
                    if k.lower() == selected:
                        matched_field = field_map[k]
                        break
                if not matched_field:
                    messagebox.showerror("Error", "Invalid field selected.", parent=win)
                    return
                cursor.execute(f"SELECT * FROM items WHERE {matched_field} LIKE %s", ('%' + keyword + '%',))

            rows = cursor.fetchall()
            tree.delete(*tree.get_children())

            for index, row in enumerate(rows):
                row = list(row)
                if len(row[9]) > 100:
                    row[9] = row[9][:100] + "..."
                tag = 'evenrow' if index % 2 == 0 else 'oddrow'
                tree.insert("", tk.END, values=row, tags=(tag,))
        except mysql.connector.Error as e:
            messagebox.showerror("Database Error", f"An error occurred: {e}", parent=win)
        finally:
            if conn.is_connected():
                cursor.close()
                conn.close()
    searchbar = customtkinter.CTkButton(
        master=SearchFrame,
        text="Search / البحث",
        command=search,
        font=("Segoe UI Semibold", 15),
        width=150,
        height=30,
        corner_radius=12,
        fg_color="#1ABC9C",
        hover_color="#16A085",
        text_color="white",
        cursor="hand2"
    )
    searchbar.place(x=315, y=3)

    cc.set("All / الكل")
    search()

import time
barcode_buffer = []
barcode_timer = None
BARCODE_TIMEOUT = 150
barcode_entry_last_time = 0
barcode_entry_timeout = 0.5  
last_keypress_time = time.time()
def scanCode(cod=None):
    barcode_buffer = []
    barcode_timer = None
    barcode_entry_last_time = 0
    barcode_entry_timeout = 0.5  

    

    BG_COLOR = "#F5F7FA"
    TEXT_COLOR = "#0C1B33"
    HEADER_COLOR = "#1F3A93"
    BTN_COLOR = "#2E8B57"
    BTN_HOVER = "#3CB371"
    FRAME_BG = "#E4E7EB"
    scanning = False

    root = customtkinter.CTk()
    root.geometry("1200x700")
    root.title("Decor Inventory App")
    root.configure(bg="#FFFFFF")
    import time
    def global_keypress(event):
        global barcode_buffer, barcode_timer, barcode_entry_last_time
        if root.focus_get() == barcode_entry:
            return

        char = event.char
        if not char.isprintable():
            return

        now = time.time()
        if now - barcode_entry_last_time > barcode_entry_timeout:
            barcode_buffer.clear()

        barcode_buffer.append(char)
        barcode_entry_last_time = now
        if barcode_timer:
            root.after_cancel(barcode_timer)
        barcode_timer = root.after(100, process_barcode_buffer)

    def process_barcode_buffer():
        global barcode_buffer, barcode_timer

        barcode = ''.join(barcode_buffer).strip()

        if barcode:
            barcode_entry.focus_set()
            barcode_entry.delete(0, tk.END)
            barcode_entry.insert(0, barcode)
            barcode_entry.event_generate("<Return>")

        barcode_buffer.clear()
        barcode_timer = None 
   

    cart = []

    def on_barcode_scanned(event=None, barcode=None):
        nonlocal scanning
        if scanning:
            return
        scanning = True
        final_barcode = barcode.strip() if barcode else barcode_entry.get().strip()
        print("Scanned Barcode:", repr(final_barcode))

        if final_barcode:
            add_item_to_cart(final_barcode)
            barcode_entry.delete(0, tk.END)

        root.after(300, reset_scan_flag)

    def reset_scan_flag():
        nonlocal scanning
        scanning = False
        barcode_entry.focus_set()
    header_frame = tk.Frame(root)
    header_frame.pack(fill="x", padx=20, pady=(20, 10))
    
    tk.Frame(root,bg= "#616467").place(x=0, y=13, width=2500, height=7)

    invoice_label = customtkinter.CTkLabel(header_frame, text="INVOICE - الفاتورة", font=("Arial", 30, "bold"), text_color="#616467")
    invoice_label.pack(side="left")

    today_date = datetime.now().strftime("%Y-%m-%d")
    new_invoice_id = 1 

    try:
        conn = mysql.connector.connect(host=os.getenv("DB_HOST","localhost"),user=os.getenv("DB_USER"),passwd=os.getenv("DB_PASSWORD"),database=os.getenv("DB_NAME"))
        cursor = conn.cursor()
        cursor.execute("SELECT MAX(id) FROM sales")
        result = cursor.fetchone()
        last_id = result[0] if result[0] else 0
        new_invoice_id = last_id + 1
        conn.close()
    except Exception as e:
        print("DB Error fetching invoice id:", e)

    cust_frame = customtkinter.CTkFrame(root)
    cust_frame.pack(fill="x", padx=20, pady=(0, 20))
    cust_frame.configure(fg_color="white")
    

# Left side customer
    labels_texts = [
    "Customer Name - أسم الزبون ",
    "Customer Phone Number - رقم هاتف الزبون",
    "Customer Paid Amount - المبلغ المدفوع",
    "Barcode - رقم المنتج"
]

    entries = []

    for i, text in enumerate(labels_texts):
        label = customtkinter.CTkLabel(cust_frame, text=text, font=("Arial", 14),text_color="#616467")
        label.grid(row=i, column=0, sticky="w", pady=5)

        entry = customtkinter.CTkEntry(cust_frame, width=300, font=("Segoe UI", 14),text_color=TEXT_COLOR)
        entry.grid(row=i, column=1, pady=5, padx=10)

        entries.append(entry)

    customer_name_entry, customer_phone_entry, customer_paid_entry, barcode_entry = entries

    
    barcode_entry.focus_set()
    barcode_entry.bind("<Return>", on_barcode_scanned)


    submit_barcode_btn = customtkinter.CTkButton(
    cust_frame,
    text="Submit",
    font=("Arial", 16, "bold"),
    fg_color=BTN_COLOR,
    hover_color=BTN_HOVER,
    
    text_color="white",
    command=on_barcode_scanned 
)
    submit_barcode_btn.grid(row=3, column=2, padx=5, pady=5)  
    submit_barcode_btn.configure(fg_color="#28A745", hover_color="#218838") 

# Set focus to barcode input after GUI loads
    root.after(100, lambda: barcode_entry.focus_set())

    # Right side invoice info
    invoice_label = customtkinter.CTkLabel(cust_frame, text="Invoice NO - رقم الفاتورة", font=("Arial", 14, "bold"),text_color="#616467")
    invoice_label.grid(row=0, column=2, sticky="w", padx=30)
    invoice_entry = customtkinter.CTkEntry(cust_frame, width=150, font=("Arial", 14))
    invoice_entry.grid(row=0, column=3, pady=5)
    invoice_entry.insert(0, str(new_invoice_id))
    invoice_entry.configure(state="disabled")

    issue_date_label = customtkinter.CTkLabel(cust_frame, text="Issue Date - تاريخ الأصدار", font=("Arial", 14, "bold"),text_color="#616467")
    issue_date_label.grid(row=1, column=2, sticky="w", padx=30)
    issue_date_entry = customtkinter.CTkEntry(cust_frame, width=150,  font=("Segoe UI", 14),text_color="#616467")
    issue_date_entry.grid(row=1, column=3, pady=5)
    issue_date_entry.insert(0, today_date)
    issue_date_entry.configure(state="disabled")
    wrapper = customtkinter.CTkFrame(root, width=2000, height=450)
    wrapper.pack(padx=0, pady=10)
    wrapper.pack_propagate(False) 

    cart_frame = customtkinter.CTkFrame(wrapper, corner_radius=15)
    cart_frame.pack(fill="both", expand=True)
    cart_frame.configure(fg_color="#FFFFFF")
    canvas = tk.Canvas(cart_frame, bg="white")
    scrollbar = tk.Scrollbar(cart_frame, orient="vertical", command=canvas.yview)
    scrollable_frame =tk.Frame(canvas,bg="white")
    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
    )

    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    scrollable_frame.configure(width=1800, height=800)
    canvas.configure(scrollregion=(0, 0, 1400, 800))

    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")
    def _on_mousewheel(event):
        canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")

    canvas.bind_all("<MouseWheel>", _on_mousewheel)
    headers = [
    ("Code - رقم المنتج", 20),
    ("Product Name - اسم المنتج", 270),
    ("Type - النوع", 600),
    ("Price - السعر", 850),
    ("Color - اللون", 1050),
    ("Quantity - الكمية", 1250),
]
    for text, xpos in headers:
        customtkinter.CTkLabel(root, text=text, font=("Arial", 14, "bold"),bg_color="#FFFFFF",text_color="black").place(x=xpos, y=252)

    widths = [150, 250, 150, 100, 150, 100, 80]
    tk.Frame(root, bg="#616467").place(x=0, y=300, width=2500, height=7)
    tk.Frame(root, bg="#616467").place(x=0, y=350, width=2500, height=7)
    total_label = customtkinter.CTkLabel(root, text="Total: 0 JD", font=("Arial", 16, "bold"))
    total_label.pack(pady=15)

    def update_total_label():
        total = 0
        for item in cart:
            try:
                price_widget = item["widget_refs"]["price"]
                if str(price_widget) and price_widget.winfo_exists():
                    price = float(price_widget.get())
                else:
                    raise Exception()
            except:
                price = float(item.get("price", 0))
            total += price * item["quantity"]
        total_label.configure(text=f"Total: {total:.2f} JD")

    def update_cart_display():
        if not root.winfo_exists():
            return
        for widget in scrollable_frame.winfo_children():
            widget.destroy()

        base_y = 80 
        row_spacing = 50  
        pre = 0
        prr = 0

        for idx, item in enumerate(cart):
            y = base_y + idx * row_spacing

            # Code
            code_lbl = customtkinter.CTkLabel(scrollable_frame, text=item["code"], width=100,text_color="#616467",font=("Segoe UI Semibold", 13))
            code_lbl.place(x=10, y=y)
            name_lbl = customtkinter.CTkLabel(scrollable_frame, text=item["name"], width=200,text_color="#616467",font=("Segoe UI Semibold", 13))
            name_lbl.place(x=240, y=y)

            type_lbl = customtkinter.CTkLabel(scrollable_frame, text=item["type"], width=150,text_color="#616467",font=("Segoe UI Semibold", 13))
            type_lbl.place(x=550, y=y)
            price_entry = customtkinter.CTkEntry(scrollable_frame, width=70,height=6,text_color="#616467",font=("Segoe UI Semibold", 13))
            price_entry.insert(0, str(item["price"]))
            price_entry.place(x=850, y=y)
            def on_price_typing(event=None, index=idx, pe=price_entry):
                try:
                    new_price = float(pe.get())
                    cart[index]["price"] = new_price
                    update_total_label()  
                except:
                    pass  

            from functools import partial
            price_entry.bind("<KeyRelease>", partial(on_price_typing, index=idx, pe=price_entry))

            color_lbl = customtkinter.CTkLabel(scrollable_frame, text=item["color"], width=150,text_color="#616467",font=("Segoe UI Semibold", 13))
            color_lbl.place(x=1010, y=y)

            # Quantity
            quantity_spin = tk.Spinbox(scrollable_frame, from_=1, to=100, width=10, font=("Arial", 12),bg="#FFFFFF")
            quantity_spin.delete(0, "end")
            quantity_spin.insert(0, item.get("quantity", 1))  # fallback to 1 if missing
            quantity_spin.place(x=1570, y=y+20+pre)
            pre+=15

            try:
                cart[idx]["quantity"] = int(quantity_spin.get())
            except:
                cart[idx]["quantity"] = 1

            def on_quantity_change_live(event=None, index=idx, qs=quantity_spin):
                try:
                    qty_text = qs.get().strip()
                    if not qty_text:
                        return  # ignore empty box until user types

                    qty = int(qty_text)

                    
                    conn = mysql.connector.connect(host=os.getenv("DB_HOST","localhost"),user=os.getenv("DB_USER"),passwd=os.getenv("DB_PASSWORD"),database=os.getenv("DB_NAME"))
                    cursor = conn.cursor()
                    cursor.execute("SELECT quantity FROM items WHERE code = %s", (cart[index]["code"],))
                    stock_quantity = cursor.fetchone()[0]
                    conn.close()

                    if qty > stock_quantity:
                        proceed = messagebox.askyesno(
                            "Stock Warning - تحذير من المخزون",
                            f"There is not enough stock for item '{cart[index]['name']}'.\n"
                            f"Available (متوفر): {stock_quantity}, Requested (مطلوب): {qty}.\n\n"
                            f"هل أنت متأكد أنك تريد إضافة المنتج للسلة رغم عدم توفر الكمية الكافية؟",
                            parent=root
                        )
                        if not proceed:
                            # Revert to previous quantity
                            qs.delete(0, "end")
                            qs.insert(0, str(cart[index]["quantity"]))
                            return

                    cart[index]["quantity"] = qty
                    update_total_label()

                except ValueError:
                    pass
                except Exception as e:
                    print("Error in quantity change:", e)

            quantity_spin.bind("<KeyRelease>", lambda e, i=idx, qs=quantity_spin: on_quantity_change_live(e, i, qs))
            quantity_spin.configure(command=lambda i=idx, qs=quantity_spin: on_quantity_change_live(index=i, qs=qs))

            # Delete button
            del_btn = customtkinter.CTkButton(scrollable_frame, text="Delete - حذف", width=60,height=6, command=lambda i=idx: delete_item(i),fg_color="#8B0000", hover_color="#A52A2A",font=("Arial", 12, "bold"))
            del_btn.place(x=1360, y=y+prr)
            prr+=2

            item["widget_refs"] = {
                "price": price_entry,
                "quantity": quantity_spin
            }
            update_total_label()


    def delete_item(index):
        if 0 <= index < len(cart):
            del cart[index]
        update_cart_display()

    def add_item_to_cart(barcode):
        try:
            conn =  mysql.connector.connect(host=os.getenv("DB_HOST","localhost"),user=os.getenv("DB_USER"),passwd=os.getenv("DB_PASSWORD"),database=os.getenv("DB_NAME"))
            cursor = conn.cursor()
            cursor.execute("SELECT code, name, type, price, color, quantity FROM items WHERE code = %s", (barcode,))
            result = cursor.fetchone()
            conn.close()
        except Exception as e:
            messagebox.showerror("Database Error, Please contact 0787655553", str(e),parent=root)
            return

        if not result:
            messagebox.showerror(
    "غير موجود - Not Found",
    "Item not found in database.\n"
    "لم يتم العثور على العنصر في قاعدة البيانات.",parent=root
)
            return

        code, name, item_type, price, color, stock_quantity = result

       
        for item in cart:
            if item["code"] == code:
                item["quantity"] += 1

                # NOW check against stock
                if item["quantity"] > stock_quantity:
                    proceed = messagebox.askyesno(
                        "Stock Warning - تحذير من المخزون",
                        f"There is not enough stock for item '{name}'. Available: {stock_quantity}, Requested: {item['quantity']}.\n"
                        f"هل أنت متأكد أنك تريد إضافة المنتج للسلة رغم عدم توفر الكمية الكافية؟",
                        parent=root
                    )
                    if not proceed:
                        item["quantity"] -= 1  
                        return
                update_cart_display()
                update_total_label()
                return

        # If not already in cart, check stock before adding new
        if 1 > stock_quantity:
            proceed = messagebox.askyesno(
                "Stock Warning - تحذير من المخزون",
                f"There is not enough stock for item '{name}'. Available: {stock_quantity}, Requested: 1.\n"
                f"هل أنت متأكد أنك تريد إضافة المنتج للسلة رغم عدم توفر الكمية الكافية؟",parent=root
            )
            if not proceed:
                return

        cart.append({
            "code": code,
            "name": name,
            "type": item_type,
            "price": price,
            "color": color,
            "quantity": 1
        })
        update_cart_display()
        update_total_label()

    def place_order():
        customer_name = customer_name_entry.get().strip()
        customer_phone = customer_phone_entry.get().strip()
        try:
            paid_amount = float(customer_paid_entry.get().strip() or 0)
        except ValueError:
            messagebox.showerror(
                "إدخال غير صالح - Invalid Input",
                "Please enter a valid number for paid amount.\n"
                "يرجى إدخال رقم صحيح للمبلغ المدفوع.",
                parent=root
            )
            return

        if not customer_name:
            messagebox.showerror(
                "معلومات ناقصة - Missing Info",
                "Please enter the customer name before placing the order.\n"
                "يرجى إدخال اسم الزبون قبل تأكيد الطلب.",
                parent=root
            )
            return

        if not cart:
            messagebox.showerror(
                "سلة فارغة - Empty Cart",
                "Cart is empty. Add items before placing an order.\n"
                "السلة فارغة. يرجى إضافة عناصر قبل تأكيد الطلب.",
                parent=root
            )
            return

        try:
            conn =  mysql.connector.connect(host=os.getenv("DB_HOST","localhost"),user=os.getenv("DB_USER"),passwd=os.getenv("DB_PASSWORD"),database=os.getenv("DB_NAME"))
            cursor = conn.cursor()

            total_amount = 0
            for item in cart:
                try:
                    price = float(item["widget_refs"]["price"].get())
                    quantity = int(item["widget_refs"]["quantity"].get())
                except Exception:
                    price = item["price"]
                    quantity = item["quantity"]

                total_amount += price * quantity

                cursor.execute("""
                    UPDATE items
                    SET quantity = quantity - %s,
                        sold = sold + %s
                    WHERE code = %s
                """, (quantity, quantity, item["code"]))

            cursor.execute("""
                INSERT INTO sales (
                    product_code, product_name, customer_name, total_amount, paid_amount, payment_method,
                    fully_paid, payment_status, date_purchased
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, CURRENT_DATE)
            """, (
                ", ".join([item["code"] for item in cart]),
                ", ".join([item["name"] for item in cart]),
                customer_name,
                total_amount,
                paid_amount,
                "Cash",
                "Yes" if paid_amount >= total_amount else "No",
                "Paid" if paid_amount >= total_amount else "Pending"
            ))

            conn.commit()
            conn.close()
        except Exception as e:
            messagebox.showerror("Database Error, Please contact 0787655553", str(e))
            return

        messagebox.showinfo(
    "نجاح - Success",
    "Order placed successfully!\n"
    "تم تأكيد الطلب بنجاح!"
)
        cart.clear()
        update_cart_display()
    update_total_label()
    def clear_cart():
        if not cart:
            messagebox.showinfo(
    "سلة فارغة - Cart Empty",
    "Cart is already empty.\n"
    "السلة فارغة بالفعل."
)
            return
        confirm = messagebox.askyesno(
    "تفريغ السلة - Clear Cart",
    "Are you sure you want to clear the entire cart?\n"
    "هل أنت متأكد أنك تريد تفريغ السلة بالكامل؟",
    parent=root
)
        if confirm:
            cart.clear()
            update_cart_display()

    clear_cart_btn = customtkinter.CTkButton(
        root,
        text="Clear Cart - تفريغ السلة",
        font=("Arial", 16, "bold"),
        fg_color="#8B0000",  
        hover_color="#A52A2A",
        text_color="white",width=6,
        command=clear_cart
    )
    clear_cart_btn.place(x=1010,y=717)
    clear_cart_btn.configure(fg_color="#B22222", hover_color="#8B0000") 

    place_order_btn = customtkinter.CTkButton(root, text="Place Order - تأكيد الطلب",font=("Arial", 16, "bold"),width=6,
    fg_color=BTN_COLOR,
    hover_color=BTN_HOVER,
    text_color="white",command=place_order)
    place_order_btn.place(x=830,y=717)
    root.bind_all("<Key>", global_keypress)
    barcode_buffer = []
    last_keypress_time = time.time()
    barcode_scan_timeout = 0.25

    def global_keypress(event):
        global barcode_buffer, last_keypress_time

        now = time.time()

        
        if now - last_keypress_time > barcode_scan_timeout:
            barcode_buffer = []

        last_keypress_time = now

        if event.char.isprintable():
            barcode_buffer.append(event.char)

        if event.keysym == "Return":
            barcode = ''.join(barcode_buffer).strip()
            barcode_buffer = []

            if barcode:
                current_widget = root.focus_get()

                if current_widget != barcode_entry:
                    try:
                        current_widget.delete(0, tk.END)  
                    except:
                        pass 

                    barcode_entry.delete(0, tk.END)
                    barcode_entry.insert(0, barcode)
                    barcode_entry.focus_set()

                    root.after(50, lambda: on_barcode_scanned(barcode=barcode))
                else:
                    on_barcode_scanned(barcode=barcode)

    root.bind_all("<Key>", global_keypress)
    root.lift()
    root.focus_force()
    barcode_entry.focus_set()


    root.mainloop()
    
HEADER_BG = "#0C1B33"
SIDEBAR_COLOR = "#0C1B33"
TEXT_COLOR = "#FAFAFA"
BUTTON_GREEN = "#2E8B57"
BUTTON_BLUE = "#33658A"
BUTTON_PURPLE = "#6A5ACD"
BUTTON_RED = "#8B0000"
BUTTON_GRAY = "#708090"
BUTTON_ORANGE = "#F39C12"
BUTTON_HOVER = "#5A9BD4"
GOLD_ACCENT = "#CBA135"
BG_COLOR = "#F4F1EA"


def center_window(win, width=1100, height=700):
    screen_width = win.winfo_screenwidth()
    screen_height = win.winfo_screenheight()
    x = (screen_width // 2) - (width // 2)
    y = (screen_height // 2) - (height // 2)
    win.geometry(f"{width}x{height}+{x}+{y}")


root = customtkinter.CTk()

screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
root.geometry(f"{screen_width}x{screen_height}+0+0")
root.attributes("-topmost", False)
root.attributes("-fullscreen", False)
canvas = tk.Canvas(master=root, width=1000, height=600, highlightthickness=0)
canvas.pack(fill="both", expand=True)

root.iconbitmap(r"C:\Users\huawei\Desktop\inventory-app\images\ChatGPT Image Jun 24, 2025, 10_18_59 AM.ico")

root.title("Decor Inventory App")
root.config(bg=BG_COLOR)
tk.Label(root, text="Dashboard Overview - نظرة عامة", font=("Segoe UI", 20, "bold"),
         bg=BG_COLOR, fg="#0C1B33").place(x=370, y=70)

MenuLogo= Image.open(r"C:\Users\huawei\Desktop\inventory-app\images\style.png")
MenuLogo=MenuLogo.resize((50,50),Image.Resampling.LANCZOS)
MenuLogo=ImageTk.PhotoImage(MenuLogo)
LeftMenu = Frame(root,bd=2,relief="ridge",bg="#616467",)
LeftMenu.place(x=0,y=0,width=300,height=1000)
tk.Label(LeftMenu, text="Navigation - التنقل", font=("Segoe UI", 16, "bold"),
         bg="#616467", fg="white").place(x=20, y=120)
header_frame = tk.Frame(LeftMenu, bg="#616467")
header_frame.place(x=20, y=23, width=250, height=80)
tk.Label(header_frame, bg="#616467", image=MenuLogo).pack(side="left", padx=5)
tk.Label(header_frame, text="Style Office", font=("Segoe UI Semibold", 20, "bold"),
         bg="#616467", fg=TEXT_COLOR).pack(side="left", padx=(0,5))


tk.Frame(LeftMenu, bg="white").place(x=0, y=105, width=350, height=2)
customtkinter.set_appearance_mode("light")  
customtkinter.set_default_color_theme("blue")
btn_width = 220
btn_height = 50
btn_x = 10
y_offset = 220 
gap = 10
image_p=r"C:\Users\huawei\Desktop\inventory-app\images\shop.png"
icon = customtkinter.CTkImage(light_image=Image.open(image_p),
                              dark_image=Image.open(image_p),
                              size=(22, 22))
b1 = customtkinter.CTkButton(master=LeftMenu,text="  Products\n  المنتجات",command=products,font=("Segoe UI", 15, "bold"),fg_color="#616467",
    corner_radius=8,
    text_color="white",
    compound="left",
    image=icon,
    anchor="w",
    width=btn_width,
    height=btn_height
)
b1.place(x=btn_x+10, y=y_offset + 0*(btn_height + gap))
image_p1=r"C:\Users\huawei\Desktop\inventory-app\images\add.png"
icon2 = customtkinter.CTkImage(light_image=Image.open(image_p1),
                              dark_image=Image.open(image_p1),
                              size=(25, 25))
b2 = customtkinter.CTkButton(
    master=LeftMenu,
    text=" Add Or Remove Item\nاضافة أو مسح منتج",
    command=open_add_window,
    font=("Segoe UI", 15, "bold"),
    fg_color="#616467",  
    hover_color="#5A9BD4",
    text_color="white",
    corner_radius=8,
    anchor="w",
    image=icon2,
    width=btn_width,
    height=btn_height
)
b2.place(x=btn_x+10, y=y_offset + 1*(btn_height + gap))
image_p2=r"C:\Users\huawei\Desktop\inventory-app\images\scan.png"
icon3 = customtkinter.CTkImage(light_image=Image.open(image_p2),
                              dark_image=Image.open(image_p2),
                              size=(42, 25))
b3 = customtkinter.CTkButton(
    master=LeftMenu,
    text=" Scan A Bar Code\nمسح ضوئي لمنتج",
    font=("Segoe UI", 15, "bold"),
    fg_color="#616467", 
    hover_color="#7B68EE",
    text_color="white",
    corner_radius=8,
    image=icon3,
    anchor="w",
    width=btn_width,
    height=btn_height,command=scanCode
)
b3.place(x=btn_x, y=y_offset + 2*(btn_height + gap))
image_p3=r"C:\Users\huawei\Desktop\inventory-app\images\expense.png"
icon4 = customtkinter.CTkImage(light_image=Image.open(image_p3),
                              dark_image=Image.open(image_p3),
                              size=(32, 32))
b5 = customtkinter.CTkButton(
    master=LeftMenu,
    text="Other Expenses\nمصاريف أخرى",
    font=("Segoe UI", 15, "bold"),
    fg_color="#616467",  
    hover_color="#778899",
    text_color="white",
    corner_radius=8,
    anchor="w",
    image=icon4,
    width=btn_width,
    height=btn_height
)
b5.place(x=btn_x+10, y=y_offset + 3*(btn_height + gap))
image_p6 = r"C:\Users\huawei\Desktop\inventory-app\images\salary.png"
icon5 = customtkinter.CTkImage(light_image=Image.open(image_p6),
                              dark_image=Image.open(image_p6),
                              size=(32, 32))
b6 = customtkinter.CTkButton(
    master=LeftMenu,
    text="Payments\nالمدفوعات",
    font=("Segoe UI", 15, "bold"),
    fg_color="#616467",      
    hover_color="#1C86EE",
    image=icon5   ,
    text_color="white",
    corner_radius=8,
    anchor="w",width=btn_width, height=btn_height
)
b6.place(x=btn_x+10, y=y_offset + 4*(btn_height + gap))
with mysql.connector.connect(host=os.getenv("DB_HOST","localhost"),user=os.getenv("DB_USER"),passwd=os.getenv("DB_PASSWORD"),database=os.getenv("DB_NAME")) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM items")
    num = cursor.fetchone()[0]
with mysql.connector.connect(host=os.getenv("DB_HOST","localhost"),user=os.getenv("DB_USER"),passwd=os.getenv("DB_PASSWORD"),database=os.getenv("DB_NAME")) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(DISTINCT type) FROM items")
    numm = cursor.fetchone()[0]
with mysql.connector.connect(host=os.getenv("DB_HOST","localhost"),user=os.getenv("DB_USER"),passwd=os.getenv("DB_PASSWORD"),database=os.getenv("DB_NAME")) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT SUM(sold) FROM items")
    total_sold = cursor.fetchone()[0] or 0
    
#SQUARES
def create_stat_card(parent, title_en, title_ar, value, color, x, y, image_path):
    frame = customtkinter.CTkFrame(master=parent, width=300, height=150, corner_radius=15, fg_color=color)
    frame.place(x=x, y=y)

    # Load and resize image
    image = Image.open(image_path)
    image = image.resize((100,100), Image.LANCZOS)
    icon = ImageTk.PhotoImage(image)
    inner = tk.Frame(master=frame, bg=color)
    inner.place(relx=0, rely=0, relwidth=1, relheight=1)

    # Image on the left
    tk.Label(inner, image=icon, bg=color).place(x=15, y=25)
    inner.image = icon  
    tk.Label(inner, text=f"{value}", font=("Segoe UI", 26, "bold"), fg="black", bg=color).place(x=310, y=30)

    tk.Label(inner, text=title_en, font=("Segoe UI", 12, "bold"), fg="black", bg=color).place(x=320, y=100,anchor="center")
    tk.Label(inner, text=title_ar, font=("Segoe UI", 12), fg="gray", bg=color).place(x=320,y=125,anchor="center")
start_x = 300
gap_x = 330
start_y = 120

create_stat_card(root, "Total Sales", "المبيعات الإجمالية", f"{total_sold}", "#FFFFFF", start_x, start_y,r"C:\Users\huawei\Desktop\inventory-app\images\saless.png")
create_stat_card(root, "Total Products", "عدد المنتجات", f"{num}", "#FFFFFF", start_x + gap_x, start_y,r"C:\Users\huawei\Desktop\inventory-app\images\inventoryy.png")
create_stat_card(root, "Categories", "عدد الفئات", f"{numm}", "#FFFFFF", start_x, start_y + 180,r"C:\Users\huawei\Desktop\inventory-app\images\categoriess.png")
create_stat_card(root, "Expenses", "المصاريف الأخرى", f"{0}", "#FFFFFF", start_x + gap_x, start_y + 180,r"C:\Users\huawei\Desktop\inventory-app\images\otherexpensess.png")






root.mainloop()
